In [ ]:
import numpy as np
import sklearn.preprocessing as preprocess
from scipy.stats import randint

## Randomized coordinate descent

### Linear system

In [ ]:
# m, n are not equal
m = 30
n = 50
mu = 0
sigma = 1

np.random.seed(1)
A = np.random.normal(mu, sigma, [m,n]) # A is m*n
A = preprocess.normalize(A,axis=1) # normalize each row of A
b = A @ np.ones(n)

In [ ]:
w, v = np.linalg.eig(A@A.T)
sigma1 = min(w)
L1 = max(w)
print(sigma1)
print(L1)

In [ ]:
epsilon = 1e-10
error_list = []
k_list = []
np.random.seed(1)
    
for i in range(100):
    x = np.random.normal(size = m)
    k = 0
    while 1>0:
        index = randint.rvs(0, m)
        d = np.zeros(m)
        d[index] = (A@A.T@x-b)[index]
        if np.linalg.norm(d/L1) < epsilon:
            break
        x = x-(1/L1)*d
        k = k+1
    w = A.T@x
    error_list.append(np.linalg.norm(A@w-b)) # error
    k_list.append(k) # iteration number
    #print(np.linalg.norm(A@w-b), k)


In [ ]:
print(np.mean(error_list))
print(np.std(error_list))
print(np.mean(k_list))
print(np.std(k_list))
#w = A.T@x
#np.linalg.norm(A@w-b),k # error and iteration numbers

### Convex function

In [ ]:
mu = 0
sigma = 1
error_list = []
k_list = []
epsilon = 1e-10

N = 500
p = 100
np.random.seed(124)
X = np.random.normal(mu, sigma, [N,p]) # X is N*p
#X = preprocess.normalize(A,axis=1) # normalize each row of A
Y = X @ np.ones(p)

w, v = np.linalg.eig(X.T@X)
sigma2 = min(w)
L2 = max(w)
print(sigma2)
print(L2)

for i in range(100):
    c = np.random.normal(size = p)
    k = 0
    while 1>0:
        index = randint.rvs(0, p)
        d = np.zeros(p)
        d[index] = (X.T@X@c-X.T@Y)[index]
        if np.linalg.norm(d/L2) < epsilon:
            break
        c = c-(1/L2)*d
        k = k+1
    error_list.append(np.linalg.norm(X@c-Y)) # error
    k_list.append(k) # iteration number
    #print(np.linalg.norm(X@c-Y),k)

In [ ]:
print(np.mean(error_list))
print(np.std(error_list))
print(np.mean(k_list))
print(np.std(k_list))

## Accelerated randomized coordinate descent

### Linear system

In [ ]:
# set up
epsilon = 1e-10
error_list = []
k_list = []
np.random.seed(2)

for i in range(100):
    x = v = np.random.normal(size = m)
    k = 0
    r = 0
    while 1>0:
        r = ((1-sigma1*r**2)/n+np.sqrt(((1-sigma1*r**2)/n)**2+4*r**2))/2
        alpha = (n-r*sigma1)/(r*(n**2-sigma1))
        beta = 1-r*sigma1/n
        y = alpha*v+(1-alpha)*x
        index = randint.rvs(0, m)
        d = np.zeros(m)
        d[index] = (A@A.T@y-b)[index]
        if np.linalg.norm(y-(1/L1)*d-x) < epsilon:
            break
        x = y-(1/L1)*d
        v = beta*v+(1-beta)*y-(r/L1)*d
        k = k+1
    w = A.T@x
    error_list.append(np.linalg.norm(A@w-b))
    k_list.append(k)
    #print(np.linalg.norm(A@w-b), k)


In [ ]:
print(np.mean(error_list))
print(np.std(error_list))
print(np.mean(k_list))
print(np.std(k_list))
#w = A.T@x
#np.linalg.norm(w-np.ones(n)),k

### Convex function

In [ ]:
# set up
epsilon = 1e-10
error_list = []
k_list = []

for i in range(100):
    x = v = np.random.normal(size = p)
    k = 0
    r = 0
    while 1>0:
        r = ((1-sigma2*r**2)/p+np.sqrt(((1-sigma2*r**2)/n)**2+4*r**2))/2
        alpha = (p-r*sigma2)/(r*(p**2-sigma2))
        beta = 1-r*sigma2/p
        y = alpha*v+(1-alpha)*x
        index = randint.rvs(0, p)
        d = np.zeros(p)
        d[index] = (X.T@X@y-X.T@Y)[index]
        if np.linalg.norm(y-(1/L2)*d-x) < epsilon:
            break
        x = y-(1/L2)*d
        v = beta*v+(1-beta)*y-(r/L2)*d
        k = k+1
    error_list.append(np.linalg.norm(X@x-Y))
    k_list.append(k)
    #print(np.linalg.norm(X@x-Y),k)

In [ ]:
print(np.mean(error_list))
print(np.std(error_list))
print(np.mean(k_list))
print(np.std(k_list))